# Linear Mixed Effects Models -- modified Edward tutorial for grouped model

### This version uses fixed effects for clusters, random effects for sites


With linear mixed effects models, we wish to model a linear
relationship for data points with inputs of varying type, categorized
into subgroups, and associated to a real-valued output.

We demonstrate with an example in Edward. A webpage version is available 
[here](http://edwardlib.org/tutorials/linear-mixed-effects-models).

In [1]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from edward.models import Normal, BernoulliWithSigmoidProbs, Bernoulli
from observations import insteval

import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy.stats import logistic


plt.style.use('ggplot')
ed.set_seed(42)

## Data: Clusters and site parameters

We will define 3 clusters, each with some number of member sites.

The data-generating model follows a very simple premise:
> Within each cluster, the sites will have a "default" or "baseline" RR (response-rate), and sites within a cluster will exhibit variation around this cluster-default.


In [2]:

# site, cluster occurrence probabilities, to generate the dataset
# as well as their "true" logit-weights 

clusters = {0: dict(prob = 0.2, w = -0.5,   # cluster weight in logistic
                    sites = np.arange(6),     
                    site_probs = [0.1, 0.1, 0.1, 0.1, 0.1, 0.5],
                    w_s =        [0.0, 0.0, 0.0, 5.0, 0.0, 0.0]),  # weights of sites in logistic
            1: dict(prob = 0.5, w = -1.0,  
                    sites = 6 + np.arange(3), 
                    site_probs = [0.1, 0.3, 0.6],
                    w_s        = [0.0, 0.0, 0.0]),                    
            2: dict(prob = 0.3, w = -0.2,  
                    sites = 9 + np.arange(3), 
                    site_probs = [0.3, 0.3, 0.4] ,
                    w_s =        [0.0, -4, 0.0 ]
                   )}


# num clusters
n_c = len(clusters)

# num sites
n_s = sum(list( map(len, list( map( lambda d: d['sites'], clusters.values()))))) # num sites

# site to cluster map
s2c = dict( [ (s,c) for c in range(n_c) for s in clusters[c]['sites']] )

# prob of each cluster occurring
p_c = [c['prob'] for c in clusters.values()]

# prob of site occurrence, within a cluster

p_s = ( [  dict( zip (d['sites'], d['site_probs']))  for d in clusters.values() ])

# "true" weights of clusters in logit model
w_c = np.array( [d['w'] for d in clusters.values()] )

# "true" weights of sites in logit model
w_s = np.concatenate( [d['w_s'] for d in clusters.values() ])


## True weights of logistic model

In [3]:
# logit for a site: sum of site-weight and its cluster-weight
def logit_site(site):
    return w_c[s2c[site]] + w_s[site]

# site response_rate
def rr_s(site):
    return logistic.cdf(logit_site(site))


### Site response-rates
Note how this matches what we wanted to model, i.e. in each cluster sites have a certain "baseline" response rate (RR), and some have much higher or much lower RR.

In [4]:
[np.round(rr_s(s),4) for s in range(n_s) ]

[0.3775,
 0.3775,
 0.3775,
 0.989,
 0.3775,
 0.3775,
 0.2689,
 0.2689,
 0.2689,
 0.4502,
 0.0148,
 0.4502]

### Data gen 
We generate N rows with `[cluster_id, site_id, abel]`

In [5]:
def gen_label(site):
    p = rr_s(site)
    return (np.random.uniform() < p)*1


def gen_row(cluster):
    site2prob = p_s[cluster]
    site_ids = list( site2prob.keys())
    probs = list(site2prob.values())
    site = site_ids[ np.random.choice(len(probs), 1, probs ) [0] ]
    return np.array( [ cluster, site, gen_label(site)] )

def gen_data(N=100):
    clusters = np.random.choice(n_c, N, list(p_c))
    return np.array(list(map(gen_row, clusters)))

train = gen_data(N=1000)
test = gen_data(N=1000)


In [6]:
train[:10]

array([[ 2, 11,  1],
       [ 0,  4,  0],
       [ 2, 11,  0],
       [ 2, 11,  0],
       [ 0,  5,  0],
       [ 0,  5,  1],
       [ 2, 11,  0],
       [ 1,  7,  0],
       [ 2, 10,  0],
       [ 2,  9,  1]])

In [7]:
c_train = train[:,0]
s_train = train[:,1]
y_train = train[:,2]
n_obs_train = train.shape[0]

c_test = test[:,0]
s_test = test[:,1]
y_test = np.array(test[:,2]).astype(np.int32)
n_obs_test = test.shape[0]

In [8]:
n_s = max(s_train) + 1  # number of sites
n_c = max(c_train) + 1  # number of clusters
n_obs = train.shape[0]  # number of observations

print("Number of sites: {}".format(n_s))
print("Number of clusters: {}".format(n_c))
print("Number of observations: {}".format(n_obs))

Number of sites: 12
Number of clusters: 3
Number of observations: 1000


## Model

Since our problem is binary classification (convert or not), we use a logistic regression where we model the _log-odds_ as a linear function of predictors.

In what follows we let $z$ denote the log-odds, and the actual prediction itself will be $1/(1+e^{-z})$.


```
z ~ (1|site) + cluster
```


In [9]:
# Set up placeholders for the data inputs.
s_ph = tf.placeholder(tf.int32, [None])
c_ph = tf.placeholder(tf.int32, [None])

# Set up random effects.

sigma_s = tf.sqrt(tf.exp(tf.get_variable("sigma_s", [])))  # random effect for a site
beta_c = tf.get_variable("sigma_c", [n_c])  # fixed effect for cluster

eta_s = Normal(loc=tf.zeros(n_s), scale=sigma_s * tf.ones(n_s))


yhat = (tf.gather(eta_s, s_ph) + # pick the entry from eta_s using site-index fed into placeholder s_ph 
        tf.gather(beta_c, c_ph))  # same thing with cluster-index fed into placeholder c_ph

# y_logit = Normal(loc=yhat, scale=tf.ones(n_obs))

y = Bernoulli(logits = yhat)



# y = tf.sigmoid(y_logit)


## Inference

Given data, we aim to infer the model's fixed and random effects.
In this analysis, we use variational inference with the
$\text{KL}(q\|p)$ divergence measure. We specify fully factorized
normal approximations for the random effects and pass in all training
data for inference. Under the algorithm, the fixed effects will be
estimated under a variational EM scheme.

In [10]:
q_eta_s = Normal(
    loc=tf.get_variable("q_eta_s/loc", [n_c]),
    scale=tf.nn.softplus(tf.get_variable("q_eta_s/scale", [n_c])))

latent_vars = {eta_s: q_eta_s}

data = {
    y: y_train,
    s_ph: s_train,
    c_ph: c_train}


inference = ed.KLqp(latent_vars, data)



TypeError: Key-value pair in latent_vars does not have same shape: (12,), (3,)

### Criticism

We will evaluate the inferred distributions by computing logits from the means of the inferred posterior distributions of the latent vars. From the logits we can compute the log-loss relative to the observed 0/1 labels

In [11]:
yhat_test = ed.copy(yhat, {eta_s: q_eta_s.mean() })


In [12]:
def log_loss(labels, probs):
    return -np.mean(labels * np.log(probs) + (1-labels)*np.log(1-probs))

def rig(labels, probs):
    p = np.mean(labels)
    ent = -p*np.log(p) - (1-p)*np.log(1-p)
    loss = log_loss(labels, probs)
    return np.round(100*(ent - loss)/ent, 2)


In [13]:
inference.initialize(n_print=2000, n_iter=20000)

tf.global_variables_initializer().run()


for _ in range(inference.n_iter):
  # Update and print progress of algorithm.
  info_dict = inference.update()

  inference.print_progress(info_dict)

  t = info_dict['t']
  if t == 1 or t % inference.n_print == 0:
    # Make predictions on test data.
    yhat_vals = yhat_test.eval(feed_dict={
        s_ph: s_test,
        c_ph: c_test})

    probs = logistic.cdf(yhat_vals)
    rg  = rig(y_test, probs)
    
    print('rig=', rg)


InvalidArgumentError: indices[0] = 11 is not in [0, 3)
	 [[Node: inference/sample/Gather_1 = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](inference/sample/sigma_c/read, _arg_Placeholder_0_0)]]

Caused by op 'inference/sample/Gather_1', defined at:
  File "/Users/pchalasani/miniconda/envs/tfbrain/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/pchalasani/miniconda/envs/tfbrain/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/pchalasani/miniconda/envs/tfbrain/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/pchalasani/miniconda/envs/tfbrain/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/pchalasani/miniconda/envs/tfbrain/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 48

In [15]:
inference_s.initialize(n_print=2000, n_iter=10000)

tf.global_variables_initializer().run()


for _ in range(inference_s.n_iter):
  # Update and print progress of algorithm.
  info_dict = inference_s.update()

  inference_s.print_progress(info_dict)

  t = info_dict['t']
  if t == 1 or t % inference.n_print == 0:
    # Make predictions on test data.
    yhat_vals = yhat_test_s.eval(feed_dict={
        s_ph: s_test})


    probs = logistic.cdf(yhat_vals)
    rg  = rig(y_test, probs)
    
    print('rig=', rg)

    1/10000 [  0%]                                ETA: 7054s | Loss: 675.057rig= -10.78
 2000/10000 [ 20%] ██████                         ETA: 9s | Loss: 575.155   rig= 12.61
 4000/10000 [ 40%] ████████████                   ETA: 6s | Loss: 578.508rig= 12.48
 6000/10000 [ 60%] ██████████████████             ETA: 3s | Loss: 579.477rig= 12.51
 8000/10000 [ 80%] ████████████████████████       ETA: 1s | Loss: 575.550rig= 12.52
10000/10000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 577.369
rig= 12.52


### Compare with MLE logistic
